https://www.metacritic.com/game 에서 크롤링 예정

In [2]:
import re
import requests
import pandas as pd
import time
from bs4 import BeautifulSoup

헤더부분 유저에이전트,레퍼러 예제

In [2]:
hur={'User-Agent':'Mozilla','referer':'https://www.metacritic.com/game/playstation-5/elden-ring'}
url='https://www.metacritic.com/game/playstation-5/elden-ring'
req=requests.get(url,headers=hur)
bsoup=BeautifulSoup(req.content,'html.parser',from_encoding='utf-8') #beautifulsoup객체선언
q1=bsoup.find("div",{"class":"details main_details"})
temp=bsoup.find("span",{"class":"blurb blurb_expanded"}).text
print(temp)

A New World Created By Hidetaka Miyazaki And George R. R. Martin. ELDEN RING, developed by FromSoftware, Inc. and BANDAI NAMCO Entertainment Inc., is a fantasy action-RPG adventure set within a world created by Hidetaka Miyazaki creator of the influential DARK SOULS video game series; and George R.R. Martin author of The New York Times best-selling fantasy series, A Song of Ice and Fire. Danger and discovery lurk around every corner in FromSoftware's largest game to-date.  Hidetaka Miyazaki, President and Game Director of FromSoftware Inc. Known for directing critically-acclaimed games in beloved franchises including Armored Core, Dark Souls, and Sekiro: Shadows Die Twice.  George R.R. Martin is the #1 New York Times bestselling author of many novels, including the acclaimed series A Song of Ice and Fire - A Game of Thrones, A Clash of Kings, A Storm of Swords, A Feast For Crows, and A Dance with Dragons. As a writer-producer, he has worked on The Twilight Zone, Beauty and the Beast, a

이전까지 크롤링했던 진도부터 하기

정해진 플랫폼에 따라 크롤링하는 함수

In [16]:
def crawl_page(url):
    ret={}
    chk=True
    print(url)
    cnt=0
    while(chk):
        req=requests.get(url,headers={'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36','referer':url}) 
        time.sleep(2.1)#크롤링 속도 제한
        if(req==None): 
            print("reqNone")
            continue
        bsoup=BeautifulSoup(req.text,'lxml')
        if(bsoup==None):
            print("bsoupNone")
            continue
        bmetascore=bsoup.find("span",{"itemprop":"ratingValue"})
        if(bmetascore==None):
            time.sleep(4)
            print("bemtaNone")
            cnt+=1;
            if(cnt>=5):
                cnt=0;
                return False;
            continue
        bsummary=bsoup.find("span",class_="blurb blurb_expanded")
        if(bsummary==None):
            print("bsumm")
            bsummary="None"
        else:
            if(bsummary.string==None):
                bsummary=bsoup.find("span",class_="blurb blurb_collapsed")
                if(bsummary.string!=None):
                    bsummary=bsummary.string
                if(bsummary.string!=None):
                    bsummary=bsummary.string
            else:
                bsummary=bsummary.string
        btitle=bsoup.find("div",{"class":"product_title"}).find("h1")
        if(btitle==None): 
            print("btitleNone")
            continue
        buserscore=bsoup.find("div",class_=re.compile("metascore_w user large game"))
        if(buserscore==None): 
            print("buserNone")
            buserscore="None"
        else:
            buserscore=buserscore.string
        bpub=bsoup.find("li",class_="summary_detail publisher")
        if(bpub==None):
            bpub="None"
        else:
            bpub=bpub.find("a").string
        bdev=(bsoup.find("li",class_="summary_detail developer"))
        if(bdev==None):
            bdev="None"
            print("bdevNone")
        else:
            bdev=bdev.find("a")
            bdev=bdev.string
        bgenres=(bsoup.find("li",class_="summary_detail product_genre"))
        if(bgenres==None):
            print("bgenreNone")
            continue
        bgenres=bgenres.find_all("span",class_="data")
        sgenres=""
        for x in range(len(bgenres)):
            sgenres=sgenres+str(bgenres[x].string)+"|"
        bgrating=bsoup.find("li",class_="summary_detail product_rating")
        if(bgrating==None):
            print("bgratingNone")
            bgrating="None"
        else:
            bgrating=bgrating.find("span",class_="data").string
        brelease=bsoup.find("li",class_="summary_detail release_data")
        brelease=brelease.find("span",class_="data")
        print(str(btitle.string)," ",str(brelease.string)," ",str(bmetascore.string)," ",str(buserscore)," ",str(bpub)[57:-53]," ",str(bdev)," ",sgenres," ",bgrating)
        ret={"title":str(btitle.string),"metascore":str(bmetascore.string),"userscore":str(buserscore),"publisher":str(bpub)[57:-53],"developer":str(bdev),"genres":sgenres,"rating":str(bgrating),"summary":str(bsummary),"release_date":str(brelease.string)}
        chk=False
    return ret
    
def get_page(url):
    req=requests.get(url,headers={'User-Agent':'Mozilla','referer':url})  
    bsoup=BeautifulSoup(req.content,'html.parser',from_encoding='utf-8')
    bsoup2=bsoup.find_all("td",{"class":"clamp-summary-wrap"})
    lpd=pd.DataFrame(columns={"title","release_date","publisher","developer","genres","rating","summary","metascore","userscore"})
    for x in bsoup2:
        y=x.find("a")["href"]
        y=y[:-15]
        h=crawl_page("https://www.metacritic.com/"+y)
        if(h==False):
            continue
        df_dic=pd.DataFrame([h])
        lpd = pd.concat([lpd, df_dic], ignore_index=True)
    return lpd
        
def crawl_direct(platform,df):
    url="https://www.metacritic.com/browse/games/release-date/available/"+platform+"/metascore?page="
    print(url)
    req=requests.get(url,headers={'User-Agent':'Mozilla','referer':url+'0'})
    bsoup=BeautifulSoup(req.content,'html.parser',from_encoding='utf-8')
    lpage_soup=bsoup.find("li",{"class":"last_page"})
    lpage2_soup=lpage_soup.find("a",{"class":"page_num"})
    lpagenum=lpage2_soup.text
    try:
        lpd=pd.read_csv(platform+'.csv')
    except:
        lpd=pd.DataFrame(columns={"title","release_date","publisher","developer","genres","rating","summary","metascore","userscore"})
    f=open(platform+"_proc.txt","r")
    proc=int(f.readline())
    f.close()
    for x in range(proc,int(lpagenum)):  
        print(url+str(x))
        page=get_page(url+str(x))
        lpd=pd.concat([page,lpd],ignore_index=True)
        lpd.to_csv(platform+'.csv',sep=',',na_rep='NaN',float_format = '%.2f',index = False)
        f=open(platform+"_proc.txt","w")
        f.write(str(x+1))
        f.close()
crawl_direct('xboxone',1)
crawl_direct('switch',1)
crawl_direct('xbox-series-x',1)

https://www.metacritic.com/browse/games/release-date/available/xboxone/metascore?page=
https://www.metacritic.com/browse/games/release-date/available/switch/metascore?page=
https://www.metacritic.com/browse/games/release-date/available/xbox-series-x/metascore?page=
https://www.metacritic.com/browse/games/release-date/available/xbox-series-x/metascore?page=0
https://www.metacritic.com//game/xbox-series-x/elden-ring
Elden Ring   Feb 25, 2022   96   7.6   Bandai Namco Games   From Software   Role-Playing|Action RPG|   M
https://www.metacritic.com//game/xbox-series-x/the-stanley-parable-ultra-deluxe
bgratingNone
The Stanley Parable: Ultra Deluxe   Apr 27, 2022   94   8.8   Galactic Cafe   Crows Crows Crows   Adventure|3D|First-Person|   None
https://www.metacritic.com//game/xbox-series-x/hades
Hades   Aug 13, 2021   93   8.6   Private Division   Supergiant Games   Action Adventure|Linear|   T
https://www.metacritic.com//game/xbox-series-x/forza-horizon-5
Forza Horizon 5   Nov  5, 2021   92

Resident Evil Village   May  7, 2021   83   8.5   Capcom   Capcom   Action Adventure|Survival|   M
https://www.metacritic.com//game/xbox-series-x/just-dance-2022
Just Dance 2022   Nov  4, 2021   83   8.3   Ubisoft   Ubisoft   Action|Rhythm|Dancing|   E10+
https://www.metacritic.com//game/xbox-series-x/oddworld-soulstorm-enhanced-edition
Oddworld: Soulstorm Enhanced Edition   Nov 30, 2021   83   7.8   Oddworld Inhabitants   Oddworld Inhabitants   Action Adventure|General|   T
https://www.metacritic.com//game/xbox-series-x/crusader-kings-iii-console-edition
Crusader Kings III: Console Edition   Mar 29, 2022   83   4.6   Paradox Interactive   Lab42   Strategy|Real-Time|General|   T
https://www.metacritic.com//game/xbox-series-x/the-elder-scrolls-v-skyrim-anniversary-edition
The Elder Scrolls V: Skyrim Anniversary Edition   Nov 11, 2021   82   3.8   Bethesda Softworks   Bethesda Game Studios   Role-Playing|Western-Style|   M
https://www.metacritic.com//game/xbox-series-x/gears-tactics
Gear

Lemnis Gate   Sep 28, 2021   78   4.0   Frontier Foundry   Ratloop Games Canada   Action|Shooter|First-Person|Arcade|   T
https://www.metacritic.com//game/xbox-series-x/iron-harvest-complete-edition
bgratingNone
Iron Harvest Complete Edition   Oct 26, 2021   78   tbd   Koch Media   KING Art   Strategy|Real-Time|Command|   None
https://www.metacritic.com//game/xbox-series-x/r-type-final-2
R-Type Final 2   Apr 30, 2021   78   5.9   NIS America   Granzella Inc.   Action|Shooter|Shoot-'Em-Up|Horizontal|   E10+
https://www.metacritic.com//game/xbox-series-x/football-manager-2021
bgratingNone
Football Manager 2021   Nov 30, 2020   78   7.4   Sega   Sports Interactive   Sports|Team|Soccer|Management|   None
https://www.metacritic.com//game/xbox-series-x/moonglow-bay
bgratingNone
Moonglow Bay   Oct  7, 2021   78   4.4   Coatsink Software   Bunnyhug   Strategy|General|Management|Business / Tycoon|   None
https://www.metacritic.com//game/xbox-series-x/fifa-22
FIFA 22   Sep 27, 2021   78   2.7   

Watch Dogs: Legion   Nov 10, 2020   74   6.2   Ubisoft   Ubisoft   Action Adventure|Open-World|   M
https://www.metacritic.com//game/xbox-series-x/loot-river
Loot River   May  3, 2022   74   tbd   SUPERHOT PRESENTS   straka.studio   Role-Playing|Roguelike|   T
https://www.metacritic.com//game/xbox-series-x/the-big-con
bgratingNone
The Big Con   Aug 31, 2021   74   tbd   Skybound Games   Mighty Yell   Adventure|General|   None
https://www.metacritic.com//game/xbox-series-x/lost-in-random
Lost in Random   Sep 10, 2021   74   7.8   Electronic Arts   Zoink!   Action Adventure|General|   E10+
https://www.metacritic.com//game/xbox-series-x/assassins-creed-valhalla-dawn-of-ragnarok
Assassin's Creed Valhalla: Dawn of Ragnarok   Mar 10, 2022   74   8.0   Ubisoft   Ubisoft Sofia   Action Adventure|Open-World|   M
https://www.metacritic.com//game/xbox-series-x/back-4-blood
Back 4 Blood   Oct  7, 2021   74   6.3   Warner Bros. Games   Turtle Rock Studios   Action|Shooter|First-Person|Arcade|   M
h

bgratingNone
Hunting Simulator 2   Mar 23, 2021   66   tbd   Maximum Games   Neopica   Sports|Individual|Nature|Hunting|   None
https://www.metacritic.com//game/xbox-series-x/stranger-of-paradise-final-fantasy-origin
Stranger of Paradise: Final Fantasy Origin   Mar 18, 2022   66   7.0   Square Enix   Team Ninja   Action Adventure|Linear|Role-Playing|Action RPG|   M
https://www.metacritic.com//game/xbox-series-x/summertime-madness
Summertime Madness   Jan 26, 2022   66   tbd   Sometimes You   DP Games   Adventure|3D|First-Person|   E
https://www.metacritic.com//game/xbox-series-x/shredders
Shredders   Mar 17, 2022   66   6.2   Foam Punch   Foam Punch   Sports|Individual|Ski / Snowboard|   E10+
https://www.metacritic.com//game/xbox-series-x/spacebase-startopia
bgratingNone
Spacebase Startopia   Mar 26, 2021   66   tbd   Kalypso   Realmforge Studios   Strategy|Real-Time|General|Command|   None
https://www.metacritic.com//game/xbox-series-x/before-we-leave
bsumm
Before We Leave   Nov 23, 2